In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
cases = pd.read_csv('data100k.csv')
cases.head(3).T

,0,1,2
person_id,102090000000110,343221000000125,343221000000125
HearingDate,2019-02-28,2009-12-07,2011-01-20
CodeSection,A.46.2-862,B.46.2-301,A.46.2-707
codesection,covered elsewhere,covered elsewhere,covered elsewhere
ChargeType,Misdemeanor,Misdemeanor,Misdemeanor
chargetype,Misdemeanor,Misdemeanor,Misdemeanor
Class,1,1,3
DispositionCode,Guilty,Guilty,Guilty
disposition,Conviction,Conviction,Conviction
Plea,NaN,NaN,NaN


In [3]:
cases['DispositionCode'].value_counts()

Guilty                     156563
Nolle Prosequi              54680
Dismissed                   42520
Guilty In Absentia          31958
Not Guilty                   5807
Not Guilty/Acquitted         1623
Not True Bill                 250
No Indictment Presented       178
Dismissed/Other                19
Name: DispositionCode, dtype: int64

In [4]:
cases['convict_binary'] = [x in ['Guilty', 'Guilty In Absentia'] for x in cases['DispositionCode']]
cases['convict_binary'].mean()

0.6421058726558083

In [5]:
cases['CodeSection'].value_counts()

A.46.2-862    26379
B.46.2-301    25967
46.2-300      17934
C.46.2-862    11728
18.2-250.1    10573
              ...  
42-51             1
14.26             1
26-51(11)         1
A46.2-870         1
18.2-95/26        1
Name: CodeSection, Length: 4207, dtype: int64

In [6]:
codeconv = cases.groupby('CodeSection').agg({'convict_binary':['count', 'mean']}).\
reset_index()

In [7]:
codeconv = codeconv.droplevel(axis=1, level=0)
codeconv = codeconv.rename({'':'CodeSection',
                           'count':'count',
                           'mean':'conv_rate'}, axis=1)
codeconv = codeconv.query("count > 100")

In [8]:
codeconv.sort_values('conv_rate', ascending=False)

,CodeSection,count,conv_rate
1755,23-22.1(A),131,0.954198
3961,A.46.2-862,26379,0.929414
3992,B.18.2-266,1879,0.905269
4012,B.46.2-357,736,0.904891
2902,46.2-357,248,0.899194
...,...,...,...
1471,19.2-358,254,0.165354
1436,19.2-123,125,0.096000
1102,18.2-374.1:1(A),112,0.089286
1433,19.2-100,238,0.000000


In [9]:
cases.groupby('Race').agg({'convict_binary': ['count', 'mean']})

convict_binary          
                                                     count      mean
Race                                                                
American Indian                                        302  0.784768
American Indian Or Alaskan Native                        1  1.000000
Asian Or Pacific Islander                             2787  0.662002
Asian or Pacific Islander                                7  0.857143
Black                                                 2200  0.604091
Black (Non-Hispanic)                                 33254  0.615926
Black(Non-Hispanic)                                  80173  0.640353
Hispanic                                              9319  0.830347
MISSING                                               1022  0.777886
Other (Includes Not Applicable.. Unknown)              615  0.562602
Other(Includes Not Applicable.. Unknown)              3452  0.764195
Unknown                                                 54  0.666667
Unknown (Includes Not Applicable.. Unknown)            785  0.727389
White                                                 3527  0.621775
White Caucasian (Non-Hispanic)                       41679  0.625279
White Caucasian(Non-Hispanic)                       114421  0.637156

In [10]:
cases['Race'].unique()

array(['Black(Non-Hispanic)', 'Hispanic', 'White Caucasian(Non-Hispanic)',
       'MISSING', 'Asian Or Pacific Islander', 'Black (Non-Hispanic)',
       'White Caucasian (Non-Hispanic)',
       'Other(Includes Not Applicable.. Unknown)',
       'Other (Includes Not Applicable.. Unknown)', 'Black', 'White',
       'Unknown (Includes Not Applicable.. Unknown)', 'American Indian',
       'Unknown', 'Asian or Pacific Islander',
       'American Indian Or Alaskan Native'], dtype=object)

In [11]:
replace_map = {'Black(Non-Hispanic)':'Black',
               'Hispanic':'Hispanic', 
               'White Caucasian(Non-Hispanic)':'White',
               'MISSING':'Unknown', 
               'Asian Or Pacific Islander':'Asian or Pacific Islander', 
               'Black (Non-Hispanic)':'Black',
               'White Caucasian (Non-Hispanic)':'White',
               'Other(Includes Not Applicable.. Unknown)':'Unknown',
               'Other (Includes Not Applicable.. Unknown)':'Unknown', 
               'Black':'Black', 
               'White':'White',
               'Unknown (Includes Not Applicable.. Unknown)':'Unknown', 
               'American Indian':'American Indian or Alaskan Native',
               'Unknown':'Unknown', 
               'Asian or Pacific Islander':'Asian or Pacific Islander',
               'American Indian Or Alaskan Native':'American Indian or Alaskan Native'}
cases['Race'] = cases['Race'].replace(replace_map)
cases["Race"].value_counts()

White                                159627
Black                                115627
Hispanic                               9319
Unknown                                5928
Asian or Pacific Islander              2794
American Indian or Alaskan Native       303
Name: Race, dtype: int64

In [12]:
cases.groupby('Race').agg({'convict_binary': ['count', 'mean']})\
.reset_index()\
.droplevel(axis=1, level=0)\
.sort_values('mean', ascending=False)

,,count,mean
3,Hispanic,9319,0.830347
0,American Indian or Alaskan Native,303,0.785479
4,Unknown,5928,0.739879
1,Asian or Pacific Islander,2794,0.662491
5,White,159627,0.633715
2,Black,115627,0.632638


In [17]:
url = 'https://demographics.coopercenter.org/sites/demographics/files/media/files/2020-07/Census_2019_RaceEstimates_forVA_0.xls'
census_race = pd.read_excel(url,skiprows=4)

,FIPS,Jurisdiction,Total Population,White Alone,Unnamed: 4,African American Alone,Unnamed: 6,Asian Alone,Unnamed: 8,Other Races Alone,Unnamed: 10,Two or more races,Unnamed: 12
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,Virginia,8535519.0,5922648.0,0.693883,1696911.0,0.198806,589710.0,0.069089,56694.0,0.006642,269556.0,0.031581
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.0,Accomack County,32316.0,21899.0,0.677652,9304.0,0.287907,257.0,0.007953,293.0,0.009067,563.0,0.017422
4,3.0,Albemarle County,109330.0,89388.0,0.817598,10600.0,0.096954,6051.0,0.055346,483.0,0.004418,2808.0,0.025684
...,...,...,...,...,...,...,...,...,...,...,...,...,...
131,800.0,Suffolk city,92108.0,47993.0,0.521051,39246.0,0.426087,1754.0,0.019043,475.0,0.005157,2640.0,0.028662
132,810.0,Virginia Beach city,449974.0,303182.0,0.673777,91161.0,0.202592,33241.0,0.073873,2907.0,0.006460,19483.0,0.043298
133,820.0,Waynesboro city,22630.0,18265.0,0.807114,3040.0,0.134335,382.0,0.016880,149.0,0.006584,794.0,0.035086
134,830.0,Williamsburg city,14954.0,11013.0,0.736458,2391.0,0.159890,904.0,0.060452,104.0,0.006955,542.0,0.036244


In [20]:
census_race = census_race.loc[~census_race['FIPS'].isnull()]
census_race.columns

Index(['FIPS', 'Jurisdiction', 'Total Population', 'White Alone', 'Unnamed: 4',
       'African American Alone', 'Unnamed: 6', 'Asian Alone', 'Unnamed: 8',
       'Other Races Alone', 'Unnamed: 10', 'Two or more races', 'Unnamed: 12'],
      dtype='object')

In [24]:
census_race = census_race[['FIPS', 'Jurisdiction', 'Total Population', 'White Alone',
       'African American Alone', 'Asian Alone']]
census_race

,FIPS,Jurisdiction,Total Population,White Alone,African American Alone,Asian Alone
3,1.0,Accomack County,32316.0,21899.0,9304.0,257.0
4,3.0,Albemarle County,109330.0,89388.0,10600.0,6051.0
5,5.0,Alleghany County,14860.0,13783.0,698.0,46.0
6,7.0,Amelia County,13145.0,10050.0,2688.0,80.0
7,9.0,Amherst County,31605.0,24299.0,6041.0,180.0
...,...,...,...,...,...,...
131,800.0,Suffolk city,92108.0,47993.0,39246.0,1754.0
132,810.0,Virginia Beach city,449974.0,303182.0,91161.0,33241.0
133,820.0,Waynesboro city,22630.0,18265.0,3040.0,382.0
134,830.0,Williamsburg city,14954.0,11013.0,2391.0,904.0


In [23]:
url = 'https://demographics.coopercenter.org/sites/demographics/files/media/files/2020-07/Census_2019_HispanicEstimates_forVA_0.xls'
census_hisp = pd.read_excel(url,skiprows=4)
census_hisp

,FIPS,Jurisdiction,"Decennial Census Count, April 1, 2010",Unnamed: 3,Unnamed: 4,"Population Estimate, July 1, 2019",Unnamed: 6,Unnamed: 7,"April 1, 2010 - July 1, 2019",Unnamed: 9
0,NaN,NaN,Total Population,Hispanic Population,NaN,Total Population,Hispanic Population,NaN,Hispanic Change,NaN
1,NaN,NaN,NaN,Total Hispanic,(%),NaN,Total Hispanic,(%),Total Hispanic,(%)
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,Virginia,8001024,631825,0.078968,8535519,834422,0.097759,202597,0.320654
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
133,800.0,"Suffolk city, Virginia",84585,2415,2.855116,92108,4300,0.046684,1885,0.780538
134,810.0,"Virginia Beach city, Virginia",437994,28987,6.618127,449974,38235,0.084972,9248,0.31904
135,820.0,"Waynesboro city, Virginia",21006,1337,6.364848,22630,1966,0.086876,629,0.470456
136,830.0,"Williamsburg city, Virginia",14068,941,6.688939,14954,1069,0.071486,128,0.136026


In [27]:
census_hisp = census_hisp.loc[~census_hisp['FIPS'].isnull()]
census_hisp.columns

Index(['FIPS', 'Jurisdiction', 'Decennial Census Count, April 1, 2010',
       'Unnamed: 3', 'Unnamed: 4', 'Population Estimate, July 1, 2019',
       'Unnamed: 6', 'Unnamed: 7', 'April 1, 2010 - July 1, 2019',
       'Unnamed: 9'],
      dtype='object')

In [29]:
census_hisp = census_hisp[['FIPS','Unnamed: 6']]
census_hisp

,FIPS,Unnamed: 6
5,1.0,2955
6,3.0,6313
7,5.0,238
8,7.0,418
9,9.0,767
...,...,...
133,800.0,4300
134,810.0,38235
135,820.0,1966
136,830.0,1069


In [34]:
census = pd.merge(census_race,census_hisp,on='FIPS',how='inner'validate='one_to_one',indicator='matched')
census

,FIPS,Jurisdiction,Total Population,White Alone,African American Alone,Asian Alone,Unnamed: 6
0,1.0,Accomack County,32316.0,21899.0,9304.0,257.0,2955
1,3.0,Albemarle County,109330.0,89388.0,10600.0,6051.0,6313
2,5.0,Alleghany County,14860.0,13783.0,698.0,46.0,238
3,7.0,Amelia County,13145.0,10050.0,2688.0,80.0,418
4,9.0,Amherst County,31605.0,24299.0,6041.0,180.0,767
...,...,...,...,...,...,...,...
128,800.0,Suffolk city,92108.0,47993.0,39246.0,1754.0,4300
129,810.0,Virginia Beach city,449974.0,303182.0,91161.0,33241.0,38235
130,820.0,Waynesboro city,22630.0,18265.0,3040.0,382.0,1966
131,830.0,Williamsburg city,14954.0,11013.0,2391.0,904.0,1069


In [35]:
census['matched'].value_counts()

both          133
left_only       0
right_only      0
Name: matched, dtype: int64

In [ ]:
census = pd.merge(census_race,census_hisp,on='FIPS',how='inner')
census

,FIPS,Jurisdiction,Total Population,White Alone,African American Alone,Asian Alone,Unnamed: 6
0,1.0,Accomack County,32316.0,21899.0,9304.0,257.0,2955
1,3.0,Albemarle County,109330.0,89388.0,10600.0,6051.0,6313
2,5.0,Alleghany County,14860.0,13783.0,698.0,46.0,238
3,7.0,Amelia County,13145.0,10050.0,2688.0,80.0,418
4,9.0,Amherst County,31605.0,24299.0,6041.0,180.0,767
...,...,...,...,...,...,...,...
128,800.0,Suffolk city,92108.0,47993.0,39246.0,1754.0,4300
129,810.0,Virginia Beach city,449974.0,303182.0,91161.0,33241.0,38235
130,820.0,Waynesboro city,22630.0,18265.0,3040.0,382.0,1966
131,830.0,Williamsburg city,14954.0,11013.0,2391.0,904.0,1069


In [38]:
census.columns

Index(['FIPS', 'Jurisdiction', 'Total Population', 'White Alone',
       'African American Alone', 'Asian Alone', 'Unnamed: 6'],
      dtype='object')

In [42]:
census = census.rename(columns={'FIPS':'fips', 
                                'Jurisdiction':'Jurisdiction', 
                                'Total Population':'population', 
                                'White Alone':'White',
                                'African American Alone':'Black', 
                                'Asian Alone':'Asian or Pacific Islander', 
                                'Unnamed: 6':'Hispanic'})
census.columns

Index(['fips', 'Jurisdiction', 'population', 'White', 'Black',
       'Asian or Pacific Islander', 'Hispanic'],
      dtype='object')

In [47]:
census_long = pd.melt(census, id_vars = ['fips', 'Jurisdiction', 'population'],
                      value_vars = ['White', 'Black',
       'Asian or Pacific Islander', 'Hispanic'])
census_long = census_long.rename(columns= {'variable':'Race','value':'racepop'})
census_long

,fips,Jurisdiction,population,Race,racepop
0,1.0,Accomack County,32316.0,White,21899.0
1,3.0,Albemarle County,109330.0,White,89388.0
2,5.0,Alleghany County,14860.0,White,13783.0
3,7.0,Amelia County,13145.0,White,10050.0
4,9.0,Amherst County,31605.0,White,24299.0
...,...,...,...,...,...
527,800.0,Suffolk city,92108.0,Hispanic,4300
528,810.0,Virginia Beach city,449974.0,Hispanic,38235
529,820.0,Waynesboro city,22630.0,Hispanic,1966
530,830.0,Williamsburg city,14954.0,Hispanic,1069


In [53]:
cases_race = cases.groupby(['CodeSection','fips','Race']).size().reset_index()
cases_race = cases_race.rename(columns={0:'race_count'})
cases_race

,CodeSection,fips,Race,race_count
0,(74-4) 26-123,550,Black,1
1,01-2007,51,White,1
2,1,550,Black,5
3,1,550,White,2
4,1,810,White,1
...,...,...,...,...
37580,Z.18.2-91; 26,700,Black,1
37581,Z.18.2-92,760,Black,1
37582,Z.18.2-95,67,Black,1
37583,Z.18.2-95,83,Black,1


In [54]:
cases_total = cases.groupby(['CodeSection','fips']).size().reset_index()
cases_total = cases_total.rename(columns={0:'total_count'})
cases_total

,CodeSection,fips,total_count
0,(74-4) 26-123,550,1
1,01-2007,51,1
2,1,550,7
3,1,810,1
4,1-12,650,75
...,...,...,...
25072,Z.18.2-91; 26,700,1
25073,Z.18.2-92,760,1
25074,Z.18.2-95,67,1
25075,Z.18.2-95,83,1


In [58]:
cases_pop = pd.merge(cases_race,cases_total, on=['CodeSection','fips'],
                     how='outer',
                     validate='many_to_one',
                     indicator='matched')
cases_pop['matched'].value_counts()

both          37585
left_only         0
right_only        0
Name: matched, dtype: int64

In [59]:
cases_pop = pd.merge(cases_race,cases_total, on=['CodeSection','fips'],
                     how='inner')
cases_pop

,CodeSection,fips,Race,race_count,total_count
0,(74-4) 26-123,550,Black,1,1
1,01-2007,51,White,1,1
2,1,550,Black,5,7
3,1,550,White,2,7
4,1,810,White,1,1
...,...,...,...,...,...
37580,Z.18.2-91; 26,700,Black,1,1
37581,Z.18.2-92,760,Black,1,1
37582,Z.18.2-95,67,Black,1,1
37583,Z.18.2-95,83,Black,1,1


In [62]:
disparity = pd.merge(cases_pop,census_long,
                    on = ['fips','Race'],
                    how = 'outer',
                    validate='many_to_one',
                    indicator='matched')
disparity['matched'].value_counts()

both          33319
left_only      4266
right_only       67
Name: matched, dtype: int64

In [63]:
disparity = pd.merge(cases_pop,census_long,
                    on = ['fips','Race'],
                    how = 'inner')
disparity 

,CodeSection,fips,Race,race_count,total_count,Jurisdiction,population,racepop
0,(74-4) 26-123,550,Black,1,1,Chesapeake city,244835.0,74950.0
1,1,550,Black,5,7,Chesapeake city,244835.0,74950.0
2,1-200,550,Black,3,3,Chesapeake city,244835.0,74950.0
3,10-10,550,Black,1,1,Chesapeake city,244835.0,74950.0
4,10-12(A),550,Black,3,3,Chesapeake city,244835.0,74950.0
...,...,...,...,...,...,...,...,...
33314,C.46.2-862,11,Asian or Pacific Islander,1,25,Appomattox County,15911.0,61.0
33315,C.46.2-862,15,Asian or Pacific Islander,1,15,Augusta County,75558.0,490.0
33316,C.46.2-862,43,Asian or Pacific Islander,1,47,Clarke County,14619.0,210.0
33317,C.46.2-862,79,Asian or Pacific Islander,3,8,Greene County,19819.0,394.0


In [66]:
disparity['RBI']  =  (disparity['race_count']/disparity['total_count'])/(disparity['racepop']/disparity['population'])
disparity

,CodeSection,fips,Race,race_count,total_count,Jurisdiction,population,racepop,RBI
0,(74-4) 26-123,550,Black,1,1,Chesapeake city,244835.0,74950.0,3.266644
1,1,550,Black,5,7,Chesapeake city,244835.0,74950.0,2.333317
2,1-200,550,Black,3,3,Chesapeake city,244835.0,74950.0,3.266644
3,10-10,550,Black,1,1,Chesapeake city,244835.0,74950.0,3.266644
4,10-12(A),550,Black,3,3,Chesapeake city,244835.0,74950.0,3.266644
...,...,...,...,...,...,...,...,...,...
33314,C.46.2-862,11,Asian or Pacific Islander,1,25,Appomattox County,15911.0,61.0,10.433443
33315,C.46.2-862,15,Asian or Pacific Islander,1,15,Augusta County,75558.0,490.0,10.28
33316,C.46.2-862,43,Asian or Pacific Islander,1,47,Clarke County,14619.0,210.0,1.481155
33317,C.46.2-862,79,Asian or Pacific Islander,3,8,Greene County,19819.0,394.0,18.863261


In [71]:
disp1 = disparity.query("total_count>=100").sort_values("RBI",ascending=False)
disp1[['46.2' not in x for x in disp1['CodeSection']]]

,CodeSection,fips,Race,race_count,total_count,Jurisdiction,population,racepop,RBI
22927,18.2-248,165,Black,30,132,Rockingham County,81948.0,2082.0,8.945507
8146,18.2-96,13,Black,77,102,Arlington County,236842.0,22922.0,7.800039
22931,18.2-250,165,Black,20,105,Rockingham County,81948.0,2082.0,7.497187
8040,14.2-81,13,Black,105,146,Arlington County,236842.0,22922.0,7.430921
8153,19.2-306,13,Black,127,183,Arlington County,236842.0,22922.0,7.170655
...,...,...,...,...,...,...,...,...,...
25737,18.2-248,760,White,4,354,Richmond city,230436.0,109958.0,0.02368
32274,18.2-388,153,Asian or Pacific Islander,1,477,Prince William County,470335.0,44429.0,0.022193
7267,18.2-57,59,Hispanic,1,281,Fairfax County,1147532.0,189515,0.021548
19479,18.2-57,153,Hispanic,1,204,Prince William County,470335.0,115173,0.020018
